# [amazon.com](https://www.amazon.com/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A16225009011%2Cn%3A172541&s=featured-rank&qid=1644647363&rnid=386442011&ref=sr_st_featured-rank) Web scraping

## I'm going to collect all useful information about top 500 (due to customers' reviews) headphones from amazon

In [1]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from tqdm import tqdm
import pandas as pd

## Opening website with selenium driver

In [38]:
s = Service("./chromedriver")
browser = Chrome(service = s)
url = "https://www.amazon.com/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A16225009011%2Cn%3A172541&s=featured-rank&qid=1644647363&rnid=386442011&ref=sr_st_featured-rank"
browser.get(url)

In [3]:
soup = BeautifulSoup(browser.page_source, "lxml")

## Finding all products cards

In [4]:
cards = soup.findAll("div", class_ = "s-card-container s-overflow-hidden aok-relative s-expand-height s-include-content-margin s-latency-cf-section s-card-border")

## Testing the first card

In [6]:
card = cards[0]

### getting link to product's page

In [25]:
link = "https://www.amazon.com"+card.find("div", class_ = "a-section").find("a").get('href')
link

'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_browse_electronics-intl-ship_sr_pg1_1?ie=UTF8&adId=A10103512VYJ7QZ2M27SM&url=%2FBulk-Classroom-Headphones-Classrooms-Comfortable%2Fdp%2FB07L4JFR3Q%2Fref%3Dsr_1_1_sspa%3Fqid%3D1644647833%26rnid%3D386442011%26s%3Delectronics%26sr%3D1-1-spons%26psc%3D1&qualifier=1644647833&id=8542828962052754&widgetName=sp_atf_browse'

### getting product's name

In [23]:
name = card.find("div", class_ = "a-section").find("h2").find("span").text.strip()

In [24]:
name

'Bulk Classroom Headphones (10 Pack) - On-Ear Premium Student Headsets: Perfect for Kids K-12 in Classrooms, Schools & Class Sets (Great Value, Durable, Noise Reducing, Comfortable Fit, Easy to Clean)'

### getting product's prices (with and without any discounts)

In [26]:
prices = card.findAll("span", class_ = "a-offscreen")

In [30]:
price = prices[0].text
price

'$99.99'

In [31]:
noDiscountPrice = prices[1].text
noDiscountPrice

'$119.99'

### getting product's rewiew rating

In [35]:
Avg_review_outof5 = card.find("span", class_ = "a-icon-alt").text.split(" ")[0]
Avg_review_outof5

'4.4'

### getting the amount of product's reviews

In [37]:
Reviews_amount = card.find("span", class_ = "a-size-base s-underline-text").text
Reviews_amount

'478'

### clicking on the "next page" button

In [39]:
browser.find_element(By.CLASS_NAME, "s-pagination-next").click()

# Final Code

In [3]:
data = []

s = Service("./chromedriver")
browser = Chrome(service = s)
url = "https://www.amazon.com/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A16225009011%2Cn%3A172541&s=featured-rank&qid=1644647363&rnid=386442011&ref=sr_st_featured-rank"
browser.get(url)

sleep(3)

soup = BeautifulSoup(browser.page_source, "lxml")

for i in tqdm(range(1,15)):
    sleep(4)
    cards = soup.findAll("div", class_ = "s-card-container s-overflow-hidden aok-relative s-expand-height s-include-content-margin s-latency-cf-section s-card-border")
    for card in cards:
        try:
            link = "https://www.amazon.com"+card.find("div", class_ = "a-section").find("a").get('href')
        except:
            link = ""
        try:
            name = card.find("div", class_ = "a-section").find("h2").find("span").text.strip()
        except:
            name =""
        try:
            price = card.findAll("span", class_ = "a-offscreen")[0].text
        except:
            price = "-"
        try:
            noDiscountPrice = card.findAll("span", class_ = "a-offscreen")[1].text
        except:
            noDiscountPrice = price
        try:
            Avg_review_outof5 = card.find("span", class_ = "a-icon-alt").text.split(" ")[0]
        except:
            Avg_review_outof5 = ""
        try:
            Reviews_amount = card.find("span", class_ = "a-size-base s-underline-text").text
        except:
            Reviews_amount = ""
        
        data.append([name, price, noDiscountPrice, Avg_review_outof5, Reviews_amount, link])
    
    try:
        browser.find_element(By.CLASS_NAME, "s-pagination-next").click()
    except:
        break

headers = ["name", "price", "noDiscountPrice", "Avg_review_outof5", "Reviews_amount", "link"]
df = pd.DataFrame(data, columns = headers)
df.to_csv("./amazon.csv", sep = ";", encoding = "utf8")

100%|██████████| 14/14 [00:57<00:00,  4.14s/it]
